In [ ]:
# 1. Install (if needed)
import sys
!{sys.executable} -m pip install umap-learn plotly pandas --quiet

# 2. Import
import numpy as np
import plotly.express as px
import pandas as pd
import umap

# 3. Load vectorstore (edit as needed)
from langchain.vectorstores import FAISS
from langchain_nomic.embeddings import NomicEmbeddings

embedding_model = NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local")
vectorstore = FAISS.load_local(
    EMBEDDINGS_DIRECTORY,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

# 4. Get all vectors and key metadata fields
dimension = vectorstore.index.d
num_vectors = vectorstore.index.ntotal
embeddings = np.vstack([vectorstore.index.reconstruct(i) for i in range(num_vectors)])

def format_metadata(m):
    # Most common fields in your art doc: title, creator, date, medium, subject
    try:
        meta = m.metadata if hasattr(m, "metadata") else {}
        title = meta.get("title") or getattr(m, "title", "") or ""
        creator = meta.get("creator") or getattr(m, "creator", "") or ""
        date = meta.get("date") or getattr(m, "date", "") or ""
        medium = meta.get("medium") or getattr(m, "medium", "") or ""
        subject = meta.get("subject") or getattr(m, "subject", "") or ""
        desc = getattr(m, "page_content", "") or ""
    except Exception:
        title = creator = date = medium = subject = desc = ""
    # Truncate long fields for hover
    short_desc = (desc[:80] + "...") if len(desc) > 80 else desc
    # You can change this structure as you want
    return f"<b>{title}</b><br>{creator}<br>{date}<br><i>{medium}</i><br>{subject}<br>{short_desc}"

try:
    metadatas = [m for m in vectorstore.docstore._dict.values()]
    tooltips = [format_metadata(m) for m in metadatas]
except Exception:
    tooltips = [f"Vector {i}" for i in range(num_vectors)]

# 5. UMAP to 3D
umap_3d = umap.UMAP(n_components=3, random_state=42)
proj_3d = umap_3d.fit_transform(embeddings)

# 6. DataFrame for Plotly
df = pd.DataFrame({
    "x": proj_3d[:,0],
    "y": proj_3d[:,1],
    "z": proj_3d[:,2],
    "tooltip": tooltips
})

# 7. Plotly 3D scatter
fig = px.scatter_3d(
    df, x="x", y="y", z="z",
    hover_name="tooltip",
    title="FAISS Vector Store Visualization (UMAP 3D, Clean Tooltips)"
)
fig.update_traces(marker=dict(size=3, opacity=0.8))
fig.update_layout(height=800, width=1200)

fig.show()


# 9. Optional: Save to HTML for standalone browser viewing (uncomment to use)
fig.write_html("vectorstore_3d.html")
print("Saved as vectorstore_3d.html — open in any browser!")
